In [5]:
from pathlib import Path
import pandas as pd
from pandas.errors import EmptyDataError

# If notebook_context set this, we don't actually need it here,
# because this notebook sits in the data folder with the CSVs.
base_dir = Path.cwd()
print("Using base_dir:", base_dir)

sources = [
    ("NBA_official_nbainjuries", base_dir / "Injury_Overrides_live_official.csv"),
    ("Balldontlie",              base_dir / "Injury_Overrides_live_balldontlie.csv"),
    ("ESPN",                     base_dir / "Injury_Overrides_live_espn.csv"),
    ("DataDriven",               base_dir / "Injury_Overrides_data_driven.csv"),
    ("Manual",                   base_dir / "Injury_Overrides.csv"),
]
# ... rest of the cell unchanged ...


frames = []
for label, path in sources:
    if not path.exists() or path.stat().st_size == 0:
        print(f"⚠️ Skipping {label}: file missing or empty at {path}")
        continue

    try:
        df = pd.read_csv(path)
    except EmptyDataError:
        print(f"⚠️ Skipping {label}: EmptyDataError in {path.name}")
        continue

    # Normalize key columns
    if "Team" not in df.columns:
        print(f"⚠️ {label}: missing 'Team' column, skipping this source.")
        continue

    # Try to normalize player column
    if "PlayerName" not in df.columns:
        if "Player" in df.columns:
            df["PlayerName"] = df["Player"]
        else:
            print(f"⚠️ {label}: missing 'PlayerName'/'Player' column, skipping.")
            continue

    # Ensure basic columns exist
    for col, default in [
        ("Status", ""),
        ("Injury", ""),
        ("EstReturn", ""),
        ("Impact", 1.0),
    ]:
        if col not in df.columns:
            df[col] = default

    # Tag with Source label (overrides whatever Source might have been)
    df["Source"] = label

    frames.append(df[["Team", "PlayerName", "Status", "Injury", "EstReturn", "Impact", "Source"]])

if not frames:
    print("❌ No usable injury data found in any source; master file will NOT be updated.")
else:
    all_inj = pd.concat(frames, ignore_index=True)

    # --- Resolve duplicates by priority ---
    # If the same (Team, PlayerName) appears in multiple sources,
    # we keep the row from the highest-priority source.
    priority_order = ["Manual", "DataDriven", "NBA_official_nbainjuries", "Balldontlie", "ESPN"]
    priority_map = {name: rank for rank, name in enumerate(priority_order, start=1)}

    all_inj["priority"] = all_inj["Source"].map(priority_map).fillna(0)

    all_inj = (
        all_inj.sort_values(["Team", "PlayerName", "priority"], ascending=[True, True, False])
        .drop_duplicates(subset=["Team", "PlayerName"], keep="first")
        .drop(columns=["priority"])
        .reset_index(drop=True)
    )

    master_path = base_dir / "Injury_Overrides_MASTER.csv"
    all_inj.to_csv(master_path, index=False)

    print("✅ Master injury file built.")
    print("   Rows in master:", len(all_inj))
    print("   Saved to:", master_path)

    display(all_inj.head(20))


Using base_dir: /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data
⚠️ Skipping Balldontlie: EmptyDataError in Injury_Overrides_live_balldontlie.csv
⚠️ Skipping ESPN: file missing or empty at /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/Injury_Overrides_live_espn.csv
⚠️ Skipping Manual: file missing or empty at /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/Injury_Overrides.csv
✅ Master injury file built.
   Rows in master: 105
   Saved to: /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project/data/Injury_Overrides_MASTER.csv


,Team,PlayerName,Status,Injury,EstReturn,Impact,Source
0,Atlanta Hawks,N'Faly Dante,mid_term_out,,,0.993596,DataDriven
1,Boston Celtics,"Harper Jr., Ron",Available,G League - Two-Way,NaN,1.000000,NBA_official_nbainjuries
2,Boston Celtics,Jayson Tatum,long_term_out,,,1.000000,DataDriven
3,Boston Celtics,"Queta, Neemias",Out,Injury/Illness - Left Ankle; Sprain,NaN,0.700000,NBA_official_nbainjuries
4,Boston Celtics,"Shulga, Max",Available,G League - Two-Way,NaN,1.000000,NBA_official_nbainjuries
5,Boston Celtics,"Tatum, Jayson",Out,Injury/Illness - Right Achilles; Repair,NaN,0.700000,NBA_official_nbainjuries
6,Brooklyn Nets,Cam Thomas,mid_term_out,,,0.922559,DataDriven
7,Charlotte Hornets,"Connaughton, Pat",Out,Injury/Illness - Right Calf; Strain,NaN,0.700000,NBA_official_nbainjuries
8,Charlotte Hornets,"Green, Josh",Out,Injury/Illness - Left Shoulder; Surgery,NaN,0.700000,NBA_official_nbainjuries
9,Charlotte Hornets,"Peterson, Drew",Out,G League - Two-Way,NaN,0.700000,NBA_official_nbainjuries
